In [1]:
# Core implementation of the proposed zero-watermarking scheme
# This notebook reproduces the main experimental pipeline reported in the paper.

In [2]:
import cv2
import os
import json
import re
import time
from imshowtools import imshow
import numpy as np
import pywt
import matplotlib.pyplot as plt
from scipy.fftpack import dct
from skimage.transform import resize
from Utility import arnoldTransform, arnoldInverseTransform, ncc, psnr,nc, calculate_psnr_color
from Utility import jpegcompression_attack,gaussian_noise,meadianfilter_attack
import imutils
import imregpoc
from utils import utils_image as util
from torchvision.utils import save_image
import secrets
import hashlib
import time
import statistics
from dataclasses import dataclass
from cryptography.hazmat.primitives.asymmetric import dsa
import matplotlib.pyplot as plt

In [3]:
folder_path = "./imgs"
output_path = "./outputs"
os_path     = "./os_img"
ms_path     = "./ms_img"
des_path     = "./descriptors"
keypoint_path     = "./keypoints"
ex_wms_path = "./extracted_wms"
attack_path = "./attacked_imgs"
reattack_path = "./re-attacked_imgs"

In [4]:
#SIGNCRYPTION
def int_to_bytes(n: int) -> bytes:
    """Encode a non-negative integer n into big-endian bytes (no leading zero byte)."""
    if n == 0:
        return b"\x00"
    length = (n.bit_length() + 7) // 8
    return n.to_bytes(length, "big")


def hash_int(n: int) -> int:
    """
    HASH(n): SHA-256(int_to_bytes(n)) → integer.
    Used for R = HASH(P) and V = HASH(M).
    """
    h = hashlib.sha256(int_to_bytes(n)).digest()
    return int.from_bytes(h, "big")


def hash_pair(R: int, S: int) -> int:
    """
    HASH(R ∥ S): used to derive Ke and Kd.
    Format: len(R)||R||len(S)||S to avoid ambiguity.
    """
    rb = int_to_bytes(R)
    sb = int_to_bytes(S)
    data = len(rb).to_bytes(2, "big") + rb + len(sb).to_bytes(2, "big") + sb
    h = hashlib.sha256(data).digest()
    return int.from_bytes(h, "big")


# =========================
# 2. SYSTEM PARAMETERS & KEYS
# =========================

@dataclass
class SystemParams:
    p: int
    q: int
    g: int


@dataclass
class UserKeys:
    sk: int  # x: secret key
    pk: int  # y: public key = g^x mod p


# ===== FAST PARAMETER & KEY GENERATION (DSA 2048/256) =====

def generate_system_params_and_key(lp: int = 2048, lq: int = 256):
    """
    Generate (p, q, g) using the DSA parameter generator from cryptography.
    - lp: bit-length of p (L)
    - lq: bit-length of q (N); for DSA with L=2048, N=256

    Returns:
        params: SystemParams(p, q, g)
        keys:   UserKeys(sk=x, pk=y)
    """
    if lp != 2048 or lq != 256:
        raise ValueError("This fast generator currently supports only L=2048, N=256 (DSA).")

    # 1) Generate DSA domain parameters (p, q, g) in C – very fast
    dsa_params = dsa.generate_parameters(key_size=lp)
    nums = dsa_params.parameter_numbers()
    p, q, g = nums.p, nums.q, nums.g

    params = SystemParams(p=p, q=q, g=g)

    # 2) Generate user key pair: x ∈ [1, q-1], y = g^x mod p
    x = secrets.randbelow(q - 1) + 1
    y = pow(g, x, p)
    keys = UserKeys(sk=x, pk=y)

    return params, keys


def generate_user_keys(params: SystemParams) -> UserKeys:
    """
    Generate a key pair for an additional user (sharing the same p, q, g).
    """
    p, q, g = params.p, params.q, params.g
    x = secrets.randbelow(q - 1) + 1
    y = pow(g, x, p)
    return UserKeys(sk=x, pk=y)


# =========================
# 4. ALGORITHM 2: ENCRYPTION
# =========================

def encrypt(params: SystemParams, xs: int, yr: int, P: int) -> tuple[int, int]:
    """
    Algorithm 2 (Encryption) – Encrypt and generate authentication tag.

    input:  p, xs, yr, P
    output: (R, C)

    Steps:
    1: Se = yr^xs mod p
    2: R  = HASH(P)
    3: Ke = HASH(R ∥ Se)
    4: C  = P * g^Ke mod p
    """
    p, g = params.p, params.g

    if not (0 < P < p):
        raise ValueError("P must be an integer in (0, p). Map plaintext into Z_p first.")

    # Step 1
    Se = pow(yr, xs, p)

    # Step 2
    R = hash_int(P)

    # Step 3
    Ke = hash_pair(R, Se)

    # Step 4
    C = (P * pow(g, Ke, p)) % p

    return R, C


# =========================
# 5. ALGORITHM 3:
#    DECRYPTION & AUTHENTICATION
# =========================

def decrypt_and_verify(params: SystemParams,
                       xr: int,
                       ys: int,
                       R: int,
                       C: int) -> tuple[int | None, bool]:
    """
    Algorithm 3 (Decryption & Authentication).

    input:  p, xr, ys, (R, C)
    output: (M, ok)

    Steps:
    1: Sd = ys^xr mod p
    2: Kd = HASH(R ∥ Sd)
    3: M  = C * g^(-Kd) mod p
    4: V  = HASH(M)
    5: If V == R  -> (M, True)
       else       -> (None, False)
    """
    p, g = params.p, params.g

    # Step 1
    Sd = pow(ys, xr, p)

    # Step 2
    Kd = hash_pair(R, Sd)

    # Step 3: g^(-Kd) mod p = (g^Kd)^(-1) mod p
    g_pow = pow(g, Kd, p)
    g_pow_inv = pow(g_pow, -1, p)  # modular inverse (Python 3.8+)

    M = (C * g_pow_inv) % p

    # Step 4
    V = hash_int(M)

    # Step 5: verify authenticity & integrity
    ok = (V == R)
    if not ok:
        return None, False
    return M, True


def os_matrix_to_int(os_matrix: np.ndarray, rows=32, cols=32) -> int:
    """
    os_matrix: 2D NumPy array with elements {0,1}, size rows x cols.
    Returns: integer P representing the bit sequence in row-major order.
    """
    # Convert to NumPy array if input is a list
    os_matrix = np.asarray(os_matrix)

    # 1. Check dimensions
    if os_matrix.shape != (rows, cols):
        raise ValueError(f"OS size must be ({rows}, {cols}), current size is {os_matrix.shape}")

    # 2. Ensure bit type and values {0,1}
    bits = os_matrix.astype(np.uint8)

    # If values are not 0/1, raise error (check via & 1)
    if not np.array_equal(bits, bits & 1):
        raise ValueError("OS must contain only bits 0 or 1")

    # 3. Row-major traversal (C-order flattening)
    flat = bits.ravel()   # length = rows * cols

    # 4. Convert to integer, first bit is MSB
    P = 0
    for b in flat:
        P = (P << 1) | int(b)

    return P


def int_to_os_matrix(P: int, rows=32, cols=32):
    """
    Convert integer P (< 2^(rows*cols)) back to a binary matrix of size rows x cols.
    """
    total_bits = rows * cols
    # Normalize to a fixed-length binary string
    bin_str = bin(P)[2:]                # remove "0b"
    bin_str = bin_str.zfill(total_bits) # pad with leading zeros

    bits = np.fromiter((int(ch) for ch in bin_str), dtype=np.uint8)
    matrix = bits.reshape((rows, cols))
    return matrix


In [5]:
# ENTROPY AND DWT-DCT CALCULATION
def dct2(block):
    return dct(dct(block.T, norm='ortho').T, norm='ortho')

def apply_dwt_dct(img):
    coeffs = pywt.dwt2(img, 'haar')
    LL, (LH, HL, HH) = coeffs
    LL_dct = dct2(LL)
    DCT_img = np.uint8(LL_dct)
    return DCT_img


def euclidean_similarity(desc1, desc2):
    v1_norm = desc1 / np.linalg.norm(desc1)
    v2_norm = desc2 / np.linalg.norm(desc2)
    dist = np.linalg.norm(v1_norm - v2_norm)
    # print("Similarity:", dist)
    return 1 / (1 + dist)  # similarity score

def _to_gray(img):
    """Ensure the input image is grayscale uint8."""
    if img is None:
        raise ValueError("Empty image (None).")
    if img.ndim == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    elif img.ndim == 2:
        gray = img
    else:
        raise ValueError("Invalid image format.")
    if gray.dtype != np.uint8:
        # Normalize to range 0..255 if not uint8
        gmin, gmax = float(gray.min()), float(gray.max())
        if gmax > gmin:
            gray = ((gray - gmin) * 255.0 / (gmax - gmin)).astype(np.uint8)
        else:
            gray = np.zeros_like(gray, dtype=np.uint8)
    return gray

def entropy_gradient_roi(img, roi, K=36, log_base=2, sobel_ksize=3):
    """
    Entropy of the gradient orientation histogram (weighted by gradient magnitude) within an ROI.
    - K: number of orientation bins (e.g., 36 → 10 degrees per bin).
    """
    x, y, w, h = map(int, roi)
    gray = _to_gray(img)
    H_img, W_img = gray.shape[:2]

    if not (0 <= x < W_img and 0 <= y < H_img):
        raise ValueError("ROI coordinates are outside the image.")
    if w <= 0 or h <= 0 or x + w > W_img or y + h > H_img:
        raise ValueError("Invalid ROI size or ROI exceeds image boundaries.")

    patch = gray[y:y+h, x:x+w].astype(np.float32)

    # Compute gradients using Sobel operator
    gx = cv2.Sobel(patch, cv2.CV_32F, 1, 0, ksize=sobel_ksize)
    gy = cv2.Sobel(patch, cv2.CV_32F, 0, 1, ksize=sobel_ksize)
    mag = np.hypot(gx, gy)
    ang = np.arctan2(gy, gx)  # range [-pi, pi]

    # Convert angle range to [0, 2*pi)
    ang = (ang + 2.0 * np.pi) % (2.0 * np.pi)

    # Orientation histogram weighted by gradient magnitude
    hist, _ = np.histogram(
        ang.ravel(),
        bins=K,
        range=(0.0, 2.0 * np.pi),
        weights=mag.ravel()
    )
    hist = hist.astype(np.float64)
    total = hist.sum()
    if total <= 0:
        return 0.0

    p = hist / total
    p = p[p > 0.0]
    H = -np.sum(p * (np.log(p) / np.log(log_base)))
    return float(H)


In [6]:
def attack_test(folder_path,save_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            print("File name: ", filename,"\n")
            # Load image
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            w, h = img.shape[:2]
            # Rotation attacks  
            print("Rotation anticlockwise attacks\n")
            for angle in [15,30,60,70,80]: 
                rotated_img = imutils.rotate(img, angle)
                new_filename = "anticlockwise_rotated_" + str(angle) + "_" + filename
                new_img_path = os.path.join(save_path,new_filename)
                cv2.imwrite(new_img_path, rotated_img)
            print("Rotation clockwise attacks\n")
            for angle in [-10,-20,-60,-70,-80]: 
                rotated_img = imutils.rotate(img, angle)
                new_filename = "clockwise_rotated_" + str(-angle) + "_" + filename
                new_img_path = os.path.join(save_path,new_filename)
                cv2.imwrite(new_img_path, rotated_img)
            print("Scaling attacks\n")
            for scale_fx in [40, 60, 90, 120,140]:
                #print (0.01 * scale_fx)
                scaled_img = cv2.resize(img, None, fx = 0.01 * scale_fx, fy = 0.01 * scale_fx)
                new_filename = "scaled_" + str(scale_fx) + "_" + filename
                new_img_path = os.path.join(save_path,new_filename)
                cv2.imwrite(new_img_path, scaled_img)
            print("Translation right attacks\n")
            for trans_x in [10,15,20,30,35]:
                delta_x = int(trans_x*0.01*img.shape[1])
                delta_y = 0
                traslated_Matrix = np.float32([[1, 0, delta_x], [0, 1, delta_y]])
                translated_img = cv2.warpAffine(img, traslated_Matrix, (img.shape[1], img.shape[0]))
                #translated_img = translate_image(img, 0.01*trans_x, trans_y, crop=False)
                new_filename = "right_translated_" + str(trans_x) + "_" + filename
                new_img_path = os.path.join(save_path,new_filename)
                cv2.imwrite(new_img_path, translated_img)
            print("Translation left attacks\n")
            for trans_x in [10,15,20,30,35]:
                delta_x = int(trans_x*0.01*img.shape[1])
                delta_y = 0
                traslated_Matrix = np.float32([[1, 0, -delta_x], [0, 1, delta_y]])
                translated_img = cv2.warpAffine(img, traslated_Matrix, (img.shape[1], img.shape[0]))
                #translated_img = translate_image(img, 0.01*trans_x, trans_y, crop=False)
                new_filename = "left_translated_" + str(trans_x) + "_" + filename
                new_img_path = os.path.join(save_path,new_filename)
                cv2.imwrite(new_img_path, translated_img)
            print("Translation down attacks\n")
            for trans_y in [7,10,15,20,25]:
                delta_y = int(trans_y*0.01*img.shape[0])
                delta_x = 0
                traslated_Matrix = np.float32([[1, 0, delta_x], [0, 1, delta_y]])
                translated_img = cv2.warpAffine(img, traslated_Matrix, (img.shape[1], img.shape[0]))
                #translated_img = translate_image(img, 0.01*trans_x, trans_y, crop=False)
                new_filename = "down_translated_" + str(trans_y) + "_" + filename
                new_img_path = os.path.join(save_path,new_filename)
                cv2.imwrite(new_img_path, translated_img)
            print("Translation up attacks\n")
            for trans_y in [10,15,20,25,30]:
                delta_y = int(trans_y*0.01*img.shape[0])
                delta_x = 0
                traslated_Matrix = np.float32([[1, 0, delta_x], [0, 1, -delta_y]])
                translated_img = cv2.warpAffine(img, traslated_Matrix, (img.shape[1], img.shape[0]))
                #translated_img = translate_image(img, 0.01*trans_x, trans_y, crop=False)
                new_filename = "up_translated_" + str(trans_y) + "_" + filename
                new_img_path = os.path.join(save_path,new_filename)
                cv2.imwrite(new_img_path, translated_img)
            print("Cropping by Y-axis attacks\n")
            for ratio in [3, 10, 15, 20,25]:
                new_height = int(h * ratio*0.01)
                crop_img = img[new_height:, :]
                new_filename = "Y_cropping_" + str(ratio) + "_" + filename
                new_img_path = os.path.join(save_path,new_filename)
                cv2.imwrite(new_img_path, crop_img)
            print("Cropping by X-axis attacks\n")
            for ratio in [3, 10, 15, 20,25]:
                new_width = int(w * ratio*0.01)
                crop_img = img[:,new_width:]
                new_filename = "X_cropping_" + str(ratio) + "_" + filename
                new_img_path = os.path.join(save_path,new_filename)
                cv2.imwrite(new_img_path, crop_img)
            print("Gaussian attacks\n")
            for ratio in [1,2,5]:
                new_img_path = os.path.join(save_path, "gaussian_"+ str(ratio) + "_"+filename)
                gaussian_img = gaussian_noise(img,mean=0, var=ratio/100.0)
                cv2.imwrite(new_img_path, gaussian_img*255.0)
            print("Jpeg attacks\n")
            for ratio in [15,25,30,40,50]:
                quality = 100-ratio
                new_img_path = os.path.join(save_path, "jpeg_"+ str(ratio) + "_"+filename.replace(".png", ".jpg"))
                jpegcompression_img = jpegcompression_attack(new_img_path, img, quality)
            print("Median filter attacks\n")
            for ksize in [3,5,7]:
                new_img_path = os.path.join(save_path, "medianfilter_"+ str(ksize) + "_"+filename)
                meadianfilter_img = meadianfilter_attack(img,ksize)
                cv2.imwrite(new_img_path, meadianfilter_img)

In [ ]:
#TempMatcher: Feature-based template matcher (ORB/AKAZE/KAZE/SIFT) 
#that loads precomputed template keypoints/descriptors, matches them to an attacked image, estimates a RANSAC homography, and outputs
#[dx,dy,rotation,scale] for geometric correction

class TempMatcher:

    def __init__(self, temp, descriptor='SIFT', filename='', numOfFeaturesPoints=50):

        # switch detector and matcher
        self.detector = self.get_des(descriptor, numOfFeaturesPoints)
        self.bf = self.get_matcher(descriptor)
        self.filename = filename

        if self.detector == 0:
            print("Unknown Descriptor! \n")
            sys.exit()

        if len(temp.shape) > 2:  # if color then convert BGR to GRAY
            temp = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)

        self.template = temp

        # build image json name exactly like your generate code intended
        img_base = os.path.basename(filename)
        img_json = os.path.splitext(img_base)[0] + ".json"

        key_dir = "./keypoints"
        des_dir = "./descriptors"

        new_kp_first = os.path.join(key_dir, "keypoint_1_" + img_json)
        new_des_first = os.path.join(des_dir, "des_1_" + img_json)

        kpfile = []
        des_list = []

        if os.path.exists(new_kp_first) and os.path.exists(new_des_first):
            for pidx in range(1, int(numOfFeaturesPoints) + 1):
                kp_path = os.path.join(key_dir, f"keypoint_{pidx}_" + img_json)
                des_path = os.path.join(des_dir, f"des_{pidx}_" + img_json)

                if not (os.path.exists(kp_path) and os.path.exists(des_path)):
                    break

                with open(kp_path, "r") as f:
                    xy = json.loads(f.read())
                x, y = float(xy[0]), float(xy[1])

                kpt = cv2.KeyPoint(
                    x=x, y=y,
                    size=1.0,
                    angle=-1.0,
                    response=0.0,
                    octave=0,
                    class_id=int(pidx)
                )
                kpfile.append(kpt)

                with open(des_path, "r") as f:
                    dv = json.loads(f.read())
                des_list.append(dv)

            if descriptor == 'ORB' or descriptor == 'AKAZE':
                desfile = np.array(des_list, dtype=np.uint8)
            else:
                desfile = np.array(des_list, dtype=np.float32)

        self.kp1 = kpfile
        self.des1 = desfile

        # homography + center
        self.H = np.eye(3, dtype=np.float32)
        self.center = np.float32([temp.shape[1], temp.shape[0]]).reshape([1, 2]) / 2
        self.inliner = 0

    def get_des(self, name, numOfFeaturesPoints):
        return {
            'ORB': cv2.ORB_create(nfeatures=numOfFeaturesPoints, scoreType=cv2.ORB_HARRIS_SCORE),
            'AKAZE': cv2.AKAZE_create(),
            'KAZE': cv2.KAZE_create(extended=False),
            'SIFT': cv2.SIFT_create()
        }.get(name, 0)

    def get_matcher(self, name):
        return {
            'ORB': cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False),
            'AKAZE': cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False),
            'KAZE': cv2.BFMatcher(),
            'SIFT': cv2.BFMatcher(),
        }.get(name, 0)

    def match(self, img, showflag=0):
        if len(img.shape) > 2:  # if color then convert BGR to GRAY
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        kp2, des2 = self.detector.detectAndCompute(img, None)
        if len(kp2) < 5:
            return [0, 0, 0, 1], 0, 0

        matches = self.bf.knnMatch(self.des1, des2, k=2)

        pts1 = []
        pts2 = []
        count = 0

        for m, n in matches:
            if m.distance < 0.5 * n.distance:
                if m.queryIdx < len(self.kp1):
                    pts2.append(kp2[m.trainIdx].pt)
                    pts1.append(self.kp1[m.queryIdx].pt)
                    count += 1

        pts1 = np.float32(pts1)
        pts2 = np.float32(pts2)

        self.inliner = 0

        if count > 4:
            self.H, self.mask = cv2.findHomography(pts1 - self.center, pts2 - self.center, cv2.RANSAC, 3.0)
            self.inliner = np.count_nonzero(self.mask)

        param = self.getpoc()
        return param, count, self.inliner

    def getpoc(self):
        Affine = self.H
        if Affine is None:
            return [0, 0, 0, 1]

        A2 = Affine * Affine
        scale = math.sqrt(np.sum(A2[0:2, 0:2]) / 2.0)
        theta = math.atan2(Affine[0, 1], Affine[0, 0])
        theta = theta * 180.0 / math.pi

        Trans = np.dot(np.linalg.inv(Affine[0:2, 0:2]), Affine[0:2, 2:3])
        return [Trans[0], Trans[1], theta, scale]

In [7]:
#Correcting_image performs feature-based registration to align an attacked image to the reference image using TempMatcher (RANSAC homography) 
#and outputs the corrected image for robust zero-watermark verification under geometric attacks.

def correcting_image(folder_path, attack_path, reattack_path,
                         ref_filename, attack_filename,
                         numOfFeaturesPoints, descriptor='SIFT'):
    # ---- load ONE reference image ----
    ref_path = os.path.join(folder_path, ref_filename)
    if not os.path.exists(ref_path):
        raise FileNotFoundError(f"Reference image not found: {ref_path}")

    ref = cv2.imread(ref_path)
    if ref is None:
        raise ValueError(f"Cannot read reference image: {ref_path}")

    # ---- load ONE attacked image ----
    comp_path = os.path.join(attack_path, attack_filename)
    if not os.path.exists(comp_path):
        raise FileNotFoundError(f"Attacked image not found: {comp_path}")

    cmp = cv2.imread(comp_path)
    if cmp is None:
        raise ValueError(f"Cannot read attacked image: {comp_path}")

    # ---- matcher (load saved keypoints/descriptors) ----
    matcher = imregpoc.TempMatcher(ref, descriptor, ref_filename, numOfFeaturesPoints)

    # keep your matching rule
    if re.findall(ref_filename, attack_filename.replace("jpg", "png")) and (
        re.findall("scaled", attack_filename) or re.findall("rotated", attack_filename) or re.findall("translated", attack_filename)
    ):
        matcho = matcher.match(cmp, 1)

        # 1) Rotation
        r_img = imutils.rotate(cmp, -1 * matcho[0][2])

        # 2) Scaling
        rs_img = cv2.resize(r_img, None, fx=1 / matcho[0][3], fy=1 / matcho[0][3])
        rs_img = cv2.resize(rs_img, (ref.shape[0], ref.shape[1]))

        # 3) Translation (your code currently disables this with 0>1, keep as-is)
        matcht = matcher.match(rs_img, 1)
        if isinstance(matcht[0][0], (np.ndarray)) and isinstance(matcht[0][1], (np.ndarray)) and 0 > 1:
            traslated_Matrix = np.float32([
                [1, 0, -1.0 * matcht[0][0][0]],
                [0, 1, -1.0 * matcht[0][1][0]]
            ])
            rst_img = cv2.warpAffine(rs_img, traslated_Matrix, (ref.shape[1], ref.shape[0]))
            out_img = rst_img
        else:
            out_img = rs_img

    elif re.findall(ref_filename, attack_filename.replace("jpg", "png")):
        # other attacks: copy as-is
        out_img = cmp
    else:
        raise ValueError(f"attack_filename does not match reference filename pattern: {attack_filename} vs {ref_filename}")

    # ---- save corrected image ----
    os.makedirs(reattack_path, exist_ok=True)
    new_filename = "re_" + attack_filename
    new_img_path = os.path.join(reattack_path, new_filename)
    cv2.imwrite(new_img_path, out_img)

    # ---- return corrected image directly ----
    return out_img

In [8]:
#constructs a zero-watermark
def generate_zero_watermark(filename, folder_path, wm_ar, N, params, xs, yr, enc_times):
    img_enc_times = []
    img_zw_times = []
    
    # Load and preprocess images
    img_path = os.path.join(folder_path, filename)
    color_img = cv2.imread(img_path)
    Ycrcb_img = cv2.cvtColor(color_img, cv2.COLOR_BGR2YCrCb)
    Y_split = Ycrcb_img[:,:,0]
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    imageSIFT = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    height = image.shape[0]
    width = image.shape[1]
    t_zw = time.perf_counter()

    # SIFT keypoint extraction
    sift = cv2.SIFT_create()
    kps, des = sift.detectAndCompute(image, None)

    # Select a region around strong keypoints, then re-rank them by ROI gradient-orientation entropy
    if len(kps) == 0:
        raise ValueError("No SIFT keypoints found!")
    idx = np.argsort([-kp.response for kp in kps])
    kp_sorted = [kps[i] for i in idx]
    des = des[idx] if des is not None else None
    for i in range(len(kp_sorted)):
        kp_sorted[i].response = 0.0
    for i in range(min(100,len(kp_sorted))):
        x, y = int(kp_sorted[i].pt[0]), int(kp_sorted[i].pt[1])
        if (x>=32) and (x<=width-32) and (y>=32) and (y<=height-32):
            roi = (x-32, y-32, 32, 32)
            new_score = entropy_gradient_roi(image, roi)
        else:
            new_score = 0.0
        kp_sorted[i].response = new_score
    idx = np.argsort([-kp.response for kp in kp_sorted])
    kp_sorted = [kp_sorted[i] for i in idx]
    des = des[idx] if des is not None else None

    p = 0
    for i in range(min(N,len(kps))):
        x, y = int(kp_sorted[i].pt[0]), int(kp_sorted[i].pt[1])
        if (x>=32) and (x<=width-32) and (y>=32) and (y<=height-32):
            p = p + 1

            # Save the selected descriptor and keypoint location for reuse
            desfile = './descriptors/'+'des_' + str(p) + "_" + filename.replace(".png",".json")
            f = open(desfile, "w")
            f.write(json.dumps(des[i].tolist()))  # JSON descriptor
            f.close()

            keypointfile = './keypoints/'+'keypoint_' + str(p) + "_" + filename.replace(".png",".json")
            f = open(keypointfile, "w")
            f.write(json.dumps([x,y]))  # JSON keypoint (x, y)
            f.close()

            # Extract a 64x64 luminance patch centered at the selected keypoint
            patch = Y_split[y-32:y+32, x-32:x+32]
            # print(patch.shape)

            # Apply DWT then DCT on the LL band
            img1 = apply_dwt_dct(patch)
            ms_img = img1.copy()

            # Binarize the transformed coefficients to form the master share (MS)
            for i in range(img1.shape[0]):
                for j in range(img1.shape[1]):
                    if (img1[i][j] >= 127):
                        ms_img[i][j] = 255
                    else: 
                        ms_img[i][j] = 0

            # plt.imshow(ms_img, cmap='Greys')
            # plt.show()
            outname = './ms_img/'+'ms_' + str(p) + "_" + filename
            cv2.imwrite(outname, ms_img)

            # Owner share (OS) obtained by XOR between MS and the watermark
            os_img = img1.copy()
            os_img_bn = img1.copy()
            for i in range(img1.shape[0]):
                for j in range(img1.shape[1]):
                    os_img[i][j] = ms_img[i][j] ^ wm_ar[i][j]
            # plt.imshow(os_img, cmap='Greys')
            # plt.show()
            
            outname = './os_img/'+'os_' + str(p) + "_" + filename
            cv2.imwrite(outname, os_img)

            # Convert OS to a binary matrix (0/1)
            os_img_bn = os_img/255

            # 3.1 Map the binary OS matrix to an integer P
            P = os_matrix_to_int(os_img_bn, rows=32, cols=32)

            if P >= params.p:
                raise ValueError(
                    "P >= p. Increase lp or reduce the OS size to ensure P < p."
                )
    
            # 3.2 Measure encryption time (Algorithm 2)
            t_enc0 = time.perf_counter()
            R, C = encrypt(params, xs, yr, P)
            t_enc1 = time.perf_counter()
            enc_times.append(t_enc1 - t_enc0)
            img_enc_times.append(t_enc1 - t_enc0)
            img_zw_times.append(t_enc1 - t_zw)

            # 3.3 Write (R, C) and P to a .txt file, separated by spaces
            RCoutname = './RCs/'+'rc_' + str(p) + "_" + filename +'.txt'
            with open(RCoutname, "w", encoding="utf-8") as f:
                f.write(f"{R} {C} {P}")

    print("Zero watermark tuples generated and saved.")
    if img_enc_times:
        enc_time = round(statistics.mean(img_enc_times) * 1000, 2)
    else:
        enc_time = 0.0  # or None, depending on your preference
    
    if img_zw_times:
        zw_time = round(statistics.mean(img_zw_times) * 1000, 2)
    else:
        zw_time = 0.0  # or None

    print("Time: ", enc_time, zw_time)
    return enc_time, zw_time


In [9]:
#extracts the zero-watermark from the input image and verifies it against the registered reference to assess authenticity under attacks
def authenticate(orgfilename,filename, folder_path,attacked_img, wm_img, N, T,key,ex_wms_path, params, xr, ys, dec_times, success_count):
    img_dec_times = []
    img_ex_times = []

    # Load and preprocess images
    org_path = os.path.join(folder_path, orgfilename)
    orgimage = cv2.imread(org_path)
    cmp_path = os.path.join("./attacked_imgs", filename.replace("re_",""))
    cmp_img = cv2.imread(cmp_path)
    image = cv2.cvtColor(attacked_img, cv2.COLOR_BGR2GRAY)
    Ycrcb_img = cv2.cvtColor(attacked_img, cv2.COLOR_BGR2YCrCb)
    Y_split = Ycrcb_img[:,:,0]
    height = image.shape[0]
    width = image.shape[1]
    t_ex = time.perf_counter()

    # SIFT keypoint extraction
    sift = cv2.SIFT_create()
    kps, des = sift.detectAndCompute(image, None)
    if len(kps) == 0:
        print("No SIFT keypoints found!")
        return 0, 0
    else:
        idx = np.argsort([-kp.response for kp in kps])
        kp_sorted = [kps[i] for i in idx]
        des = des[idx] if des is not None else None
        for i in range(len(kp_sorted)):
            kp_sorted[i].response = 0.0
        for i in range(min(100,len(kp_sorted))):
            x, y = int(kp_sorted[i].pt[0]), int(kp_sorted[i].pt[1])
            if (x>=32) and (x<=width-32) and (y>=32) and (y<=height-32):
                roi = (x-32, y-32, 32, 32)
                new_score = entropy_gradient_roi(image, roi)
            else:
                new_score = 0.0
            kp_sorted[i].response = new_score
        idx = np.argsort([-kp.response for kp in kp_sorted])
        kp_sorted = [kp_sorted[i] for i in idx]
        des = des[idx] if des is not None else None

        print(filename)

        # Select candidate ROIs, then match each ROI descriptor against stored descriptors
        q = 0
        W = []
        for i in range(min(N,len(kp_sorted))):
            x, y = int(kp_sorted[i].pt[0]), int(kp_sorted[i].pt[1])
            if (x>=32) and (x<=width-32) and (y>=32) and (y<=height-32):
                max_similarity = 0
                # max_distance = 0
                index = 0

                # Find the most similar stored descriptor for the corresponding original image
                for desfilename in os.listdir('./descriptors'):
                    if re.findall(orgfilename.replace(".png",".json"),desfilename):
                        f = open('./descriptors/'+desfilename, "r")
                        rawDes = json.loads(f.read())
                        f.close()
                        desfile = np.array(rawDes,dtype=np.float32)
                        similarity = euclidean_similarity(des[i],desfile)
                        if max_similarity < similarity:  # and distance <= 2:
                            max_similarity = similarity
                            index = desfilename.split("_")[1]

                if max_similarity >= T:
                    q = q + 1

                    # Extract a 64x64 luminance patch centered at the matched keypoint
                    patch = Y_split[y-32:y+32, x-32:x+32]
                    img1 = apply_dwt_dct(patch)
                    ms_img = img1
                    for i in range(img1.shape[0]):
                        for j in range(img1.shape[1]):
                            if (img1[i][j] >= 127):
                                ms_img[i][j] = 255
                            else: 
                                ms_img[i][j] = 0

                    # 3.3 Measure decryption + authentication time (Algorithm 3)
                    # Read (R, C) from the corresponding file
                    rc_path = './RCs/'+'rc_' + str(index) + "_" + orgfilename+'.txt'
                    with open(rc_path, "r", encoding="utf-8") as f:
                        R, C, P = map(int, f.readline().split())
                        # print(R, C, P);

                    t_dec0 = time.perf_counter()
                    M, ok = decrypt_and_verify(params, xr, ys, R, C)
                    t_dec1 = time.perf_counter()
                    dec_times.append(t_dec1 - t_dec0)
                    img_dec_times.append(t_dec1 - t_dec0)

                    if ok and M == P:
                        success_count += 1
                        # print("Encrypted authentication succeeded!")
                    else:
                        print("Encrypted authentication FAILED!")

                    os_img_bn = int_to_os_matrix(M, rows=32, cols=32) if ok else None
                    os_img = 255 * os_img_bn
                    # plt.imshow(os_img, cmap='Greys')
                    # plt.show()

                    wm_ar_xor = img1
                    for i in range(img1.shape[0]):
                        for j in range(img1.shape[1]):
                            wm_ar_xor[i][j] = ms_img[i][j] ^ os_img[i][j]

                    # Apply inverse Arnold transform to recover the extracted watermark candidate
                    wm_ar_inv1 = arnoldInverseTransform(wm_ar_xor, key)
                    W.append(wm_ar_inv1)

        # Majority vote fusion over all extracted watermark candidates
        Wq = np.zeros((32, 32), dtype=np.uint8)
        # print(Wq)
        for i in range(Wq.shape[0]):
            for j in range(Wq.shape[1]):
                totalij = 0
                for k in range(len(W)):
                    totalij = totalij + W[k][i][j]/255
                if totalij >= len(W)/2:
                    Wq[i][j] = 255
                else:
                    Wq[i][j] = 0

        t_ex1 = time.perf_counter()
        ex_time = round((t_ex1 - t_ex)*1000,2)

        if img_dec_times:  # non-empty list
            dec_time = round(statistics.mean(img_dec_times) * 1000, 2)
        else:
            dec_time = 0.0  # or None, depending on your preference

        ex_filename = "ex_" + filename
        new_ex_path = os.path.join(ex_wms_path, ex_filename)
        cv2.imwrite(new_ex_path, Wq)

        # Calculate NC
        # print("W:", wm_img)
        # print("W':", Wq)
        resNC = round(nc(wm_img,Wq),3)
        print ("NC = %.3f" %resNC)

        # Calculate PSNR
        resPSNR = round(calculate_psnr_color(orgimage, cv2.resize(cmp_img, (orgimage.shape[0], orgimage.shape[1]))),3)
        print ("PSNR = %.3f" %resPSNR)

        # imshow(orgimage, attacked_img, wm_img, Wq, mode='BGR')
        return resPSNR, resNC, dec_time, ex_time


In [10]:
def runExperiments(N,T, params, xs, yr, enc_times, xr, ys, dec_times, success_count):
    for orgfilename in os.listdir(folder_path):
        if orgfilename.endswith(".jpg") or orgfilename.endswith(".png") or orgfilename.endswith(".bmp"):
            print(orgfilename)
            #Generate zero watermark
            enc_time,zw_time = generate_zero_watermark(orgfilename,folder_path, wm_ar,N, params, xs, yr, enc_times)
          
            for filename in os.listdir(attack_path):
                if re.findall(orgfilename,filename.replace(".jpg",".png")):# and re.findall("gaussian_",filename) :
                    print(orgfilename,filename.replace(".jpg",".png"))
                    # Correcting attacked image before authentication
                    correctedimg = correcting_image(folder_path, attack_path, reattack_path,orgfilename, filename,N, descriptor='SIFT')
                    # Verification
                    xPSNR, xNC, dec_time, ex_time = authenticate(orgfilename,filename, folder_path,correctedimg, wm_img, N, T,key,ex_wms_path, params, xr, ys, dec_times, success_count)


In [11]:
wm_img = cv2.imread("./wms/wm.png", 0)
_, wm_img = cv2.threshold(wm_img, 127, 255, cv2.THRESH_BINARY)
wm_img.shape
# arnoldTransform process
a = 6
b = 40
key = 33
wm_ar = arnoldTransform(wm_img, key)

In [12]:
# BENCHMARKING THE ALGORITHM
lp = 2048
lq = 256
w, h = wm_img.shape[:2]
print("=== START BENCHMARKING THE ALGORITHM ===")
print(f"lp = {lp} bit, lq = {lq} bit, OS = {w}x{h} bit")

# 1. Measure the time for system parameter generation and key generation for the sender
t0 = time.perf_counter()
params, sender_keys = generate_system_params_and_key(lp, lq)
t1 = time.perf_counter()
keygen_time_ms = (t1 - t0) * 1000

# 2. Generate keys for the receiver (using the same system parameters)
t2 = time.perf_counter()
receiver_keys = generate_user_keys(params)
t3 = time.perf_counter()
user_keygen_time_ms = (t3 - t2) * 1000

print(f"- Time to generate (p, q, g, x_s, y_s): {keygen_time_ms:.2f} ms")
print(f"- Time to generate keys for another user (x_r, y_r): {user_keygen_time_ms:.2f} ms")

xs, ys = sender_keys.sk, sender_keys.pk
xr, yr = receiver_keys.sk, receiver_keys.pk

# 3. Measure encryption / decryption time over multiple OS instances
enc_times = []
dec_times = []
success_count = 0

# Attack images:
attack_test(folder_path, attack_path)

# Zero-watermark generation and authentication
# Write results to an Excel file

# P: number of strongest extracted keypoints and their descriptors
# for P in [40, 50, 60, 80, 100]:
for P in [80]:
    for filename in os.listdir(ms_path):
        file_path = os.path.join(ms_path, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)
    for filename in os.listdir(os_path):
        file_path = os.path.join(os_path, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)
    for filename in os.listdir(des_path):
        file_path = os.path.join(des_path, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)
    for filename in os.listdir(keypoint_path):
        file_path = os.path.join(keypoint_path, filename)
        if os.path.isfile(file_path):
            os.remove(file_path)

    # for T in [0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]:
    for T in [0.65]:
        print("Parameter set (P, T): ", P, T)
        runExperiments(P, T, params, xs, yr, enc_times, xr, ys, dec_times, success_count)

# 4. Statistics
enc_times_ms = [t * 1000 for t in enc_times]
dec_times_ms = [t * 1000 for t in dec_times]

# Encryption times
if enc_times_ms:
    avg_enc = statistics.mean(enc_times_ms)
    std_enc = statistics.pstdev(enc_times_ms) if len(enc_times_ms) > 1 else 0.0
else:
    avg_enc = 0.0   # or None, depending on preference
    std_enc = 0.0

# Decryption times
if dec_times_ms:
    avg_dec = statistics.mean(dec_times_ms)
    std_dec = statistics.pstdev(dec_times_ms) if len(dec_times_ms) > 1 else 0.0
else:
    avg_dec = 0.0   # or None
    std_dec = 0.0

print("\n=== PERFORMANCE RESULTS ===")
# print(f"- Number of test rounds: {n_rounds}")
# print(f"- Number of successful decryptions & authentications: {success_count}/{n_rounds}")

print(f"- Average encryption time: {avg_enc:.4f} ms "
      f"(standard deviation ≈ {std_enc:.4f} ms)")
print(f"- Average decryption time: {avg_dec:.4f} ms "
      f"(standard deviation ≈ {std_dec:.4f} ms)")

# (Optional) Throughput estimation: OS per second
if avg_enc > 0:
    enc_throughput = 1000.0 / avg_enc
    print(f"- Approximate encryption throughput: {enc_throughput:.2f} OS/s")
if avg_dec > 0:
    dec_throughput = 1000.0 / avg_dec
    print(f"- Approximate decryption throughput: {dec_throughput:.2f} OS/s")


=== START BENCHMARKING THE ALGORITHM ===
lp = 2048 bit, lq = 256 bit, OS = 32x32 bit
- Time to generate (p, q, g, x_s, y_s): 1940.69 ms
- Time to generate keys for another user (x_r, y_r): 8.90 ms
File name:  house.png 

Rotation anticlockwise attacks

Rotation clockwise attacks

Scaling attacks

Translation right attacks

Translation left attacks

Translation down attacks

Translation up attacks

Cropping by Y-axis attacks

Cropping by X-axis attacks

Gaussian attacks

Jpeg attacks

Median filter attacks

File name:  img1.png 

Rotation anticlockwise attacks

Rotation clockwise attacks

Scaling attacks

Translation right attacks

Translation left attacks

Translation down attacks

Translation up attacks

Cropping by Y-axis attacks

Cropping by X-axis attacks

Gaussian attacks

Jpeg attacks

Median filter attacks

Parameter set (P, T):  80 0.65
house.png
Zero watermark tuples generated and saved.
Time:  15.67 1453.9
house.png anticlockwise_rotated_15_house.png
anticlockwise_rotated_15_

KeyboardInterrupt: 